# Learn the motion from data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import roslib
import tf
from memmo_utils import *
from database import *
import os
from geometry_msgs.msg import Quaternion, Pose, Point, Vector3
from std_msgs.msg import Header, ColorRGBA
import rospy
import os
from mlp.utils.status import Status
import mlp.utils.wholebody_result as wb_res
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output

from transforms3d import axangles
from transforms3d import affines, quaternions

#from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
#from pykdl_utils.kdl_kinematics import KDLKinematics


from regression import *

np.set_printoptions(precision=4, suppress=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline


### Start Instruction

### Load Robot using URDF 

In [ ]:
import roslib; roslib.load_manifest('urdfdom_py')
from urdf_parser_py.urdf import URDF
robot = URDF.from_parameter_server()
clear_output()

tree = kdl_tree_from_urdf_model(robot)

left_foot = KDLKinematics(robot, 'base_link', 'left_sole_link')
right_foot = KDLKinematics(robot, 'base_link', 'right_sole_link')

q = left_foot.random_joint_angles()
pose = left_foot.forward(q) # forward kinematics (returns homogeneous 4x4 numpy.mat)
q_ik = left_foot.inverse(pose, q+0.3) # inverse kinematics

### Create Visualization

In [ ]:
viz = Visual()
viz.rate = rospy.Rate(1000)

foot_marker = RvizMarker('foot_marker', 5, Marker.CUBE, 4)
colors = [ColorRGBA(0.0, 1.0, 0.0, 0.7), ColorRGBA(0.0, 0.0, 1.0, 0.7)] + [ColorRGBA(1.0, 0.0, 0.0, 0.9)]*2
foot_marker.set_color(colors)

### Loading Files

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'

In [ ]:
use_crocs = False
if use_crocs:
    f = open(ROOT_CROCS + '/data_left_right.pkl', 'rb')
else:
    f = open(ROOT_PROCESSED + '/data_left_right.pkl', 'rb')
data = pickle.load(f)
f.close()

trajs = dict()
vel_trajs = dict()
x_inputs = dict()
foot_poses = dict()

In [ ]:
goal_type = 'contact'#'root'#
num_desired = 600
keys = ['left','right']
num_data = dict()
for key in keys:
    trajs[key] = np.array(data[key]['trajs'])[:num_desired]
    vel_trajs[key] = np.array(data[key]['vel_trajs'])[:num_desired]
    if goal_type == 'contact':
        x_inputs[key] = np.array(data[key]['x_inputs'])[:num_desired]
    elif goal_type == 'root':
        x_inputs[key] = np.array(data[key]['x_inputs_root'])[:num_desired]
    foot_poses[key] = data[key]['foot_poses'][:num_desired]
    num_data[key] = len(foot_poses[key])

#### calculate the control trajectory 

In [ ]:
u_trajs = dict()
for key in keys:
    if use_crocs:
        raw_trajs = data[key]['u_trajs']
        for i in range(len(raw_trajs)):
            raw_trajs[i][59] = raw_trajs[i][58]
        u_trajs[key] = np.array(raw_trajs)
    else:
        u_trajs[key] = np.array(data[key]['u_trajs'])[:num_desired]

#### Check for error 

In [ ]:
if use_crocs:
    del_indexes = dict()
    for key in keys:
        del_indexes[key] = []
        for i in range(num_data[key]):
            if np.max(np.abs(trajs[key][i])) > 1e2:
                del_indexes[key] += [i]

#### Delete the error data 

In [ ]:
if use_crocs:
    for key in keys:
        trajs[key] = np.delete(trajs[key],del_indexes[key],axis=0)
        x_inputs[key] = np.delete(x_inputs[key],del_indexes[key],axis=0)
        foot_poses[key] = np.delete(foot_poses[key],del_indexes[key],axis=0)
        vel_trajs[key] = np.delete(vel_trajs[key],del_indexes[key],axis=0)
        u_trajs[key] = np.delete(u_trajs[key],del_indexes[key],axis=0)    

#### Subsample to the original time intervals (1ms) 

In [ ]:
if use_crocs:
    original_T = 3351
    #original_T = 4351

    if use_crocs:
        new_trajs = dict()
        new_vel_trajs = dict()
        new_u_trajs = dict()
        for key in keys:
            new_trajs[key] = []
            new_vel_trajs[key] = []
            new_u_trajs[key] = []
            for i in range(len(trajs[key])):
                traj = trajs[key][i]
                new_traj = subsample(traj, original_T)
                new_trajs[key].append(new_traj)

                vel_traj = vel_trajs[key][i]
                new_vel_traj = subsample(vel_traj, original_T)
                new_vel_trajs[key].append(new_vel_traj)

                u_traj = u_trajs[key][i]
                new_u_traj = subsample(u_traj, original_T)
                new_u_trajs[key].append(new_u_traj)

            new_trajs[key] = np.array(new_trajs[key])
            new_vel_trajs[key] = np.array(new_vel_trajs[key])
            new_u_trajs[key] = np.array(new_u_trajs[key])

        trajs = new_trajs
        vel_trajs = new_vel_trajs
        u_trajs = new_u_trajs

In [ ]:
viz.set_rate(1000)

key = 'left'
for i in range(1):
    foot_marker.publish(x_inputs[key][i].reshape(-1,3))
    viz.vis_traj(trajs[key][i])

### Define RBF 

In [ ]:
timestep = 3351#trajs['left'].shape[1]
rbf_num = 60
Phi = define_RBF(dof=39, nbStates = rbf_num, offset = 200, width = 60, T = timestep)
%matplotlib qt
plt.plot(Phi)
plt.savefig('/home/rli/git/presentations/MEMMO/2019_sab_meeting/figures/rbf.png')
#clear_output()

#### Apply RBF 

In [ ]:
def apply_RBF(trajs, Phi, rcond=0.0001):
    w_trajs = []
    for traj in trajs:
        w,_,_,_ = np.linalg.lstsq(Phi, traj, rcond=0.0001)
        w_trajs.append(w.flatten())
    return np.array(w_trajs)
    
def inverse_transform(w_pca, pca, Phi, rbf_num):
    w = pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    traj = np.dot(Phi,w)
    return traj

In [ ]:
w_trajs = dict()
w_vel_trajs = dict()
w_u_trajs = dict()

for key in keys:
    w_trajs[key] = apply_RBF(trajs[key], Phi)
    if use_crocs:
        w_vel_trajs[key] = apply_RBF(vel_trajs[key], Phi)
        w_u_trajs[key] = apply_RBF(u_trajs[key], Phi)

### Visualize the approximated trajectories 

#### Try PCA decomposition 

In [ ]:
trajs['left'].shape
w_trajs2 = dict()
w_trajs2['left'] = trajs['left'].reshape(600,-1)
w_trajs2['right'] = trajs['right'].reshape(600,-1)

from sklearn.decomposition import PCA
pca = PCA(n_components=40)
import time
tic = time.time()
w_trajs_pca = dict()
w_trajs_pca['left'] = pca.fit_transform(w_trajs2['left'])
w_trajs_pca['right'] = pca.fit_transform(w_trajs2['right'])
toc = time.time()
print toc-tic

In [ ]:
from sklearn.decomposition import PCA

num_pca_comp = 60

w_trajs_pca = dict()
pca = dict()

w_vel_trajs_pca = dict()
pca_vel = dict()

w_u_trajs_pca = dict()
pca_u = dict()

for key in keys:
    pca[key] = PCA(n_components=num_pca_comp)
    w_trajs_pca[key] = pca[key].fit_transform(w_trajs[key])
    
    if use_crocs:
        pca_vel[key] = PCA(n_components=num_pca_comp)
        w_vel_trajs_pca[key] = pca_vel[key].fit_transform(w_vel_trajs[key])

        pca_u[key] = PCA(n_components=num_pca_comp)
        w_u_trajs_pca[key] = pca_u[key].fit_transform(w_u_trajs[key])

In [ ]:
key = 'left'
for i in range(10):
    w_pca = w_trajs_pca[key][i]
    w = pca[key].inverse_transform(w_pca)
    w = w.reshape(60,-1)
    #traj = w.reshape(3351,-1)
    traj = np.dot(Phi,w)
    foot_marker.publish(foot_poses[key][i])
    viz.vis_traj(traj)
    raw_input()

In [ ]:
key = 'left'
for i in range(10):
    w_pca = w_trajs_pca[i]
    w = pca.inverse_transform(w_pca)
    traj = w.reshape(3351,-1)
    foot_marker.publish(foot_poses[key][i])
    viz.vis_traj(traj)
    raw_input()

### Learning from Data 

In [ ]:
from regression import *

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_inputs_train = dict()
x_inputs_test = dict()
y_train = dict()
y_test = dict()

y_vel_train = dict()
y_vel_test = dict()

y_u_train = dict()
y_u_test = dict()

for key in keys:
    x_inputs_train[key], x_inputs_test[key], y_train[key], y_test[key] = train_test_split(x_inputs[key],w_trajs_pca[key], test_size = 0.1666, random_state=1)
    if use_crocs:
        _,_, y_vel_train[key], y_vel_test[key] = train_test_split(x_inputs[key],w_vel_trajs_pca[key], test_size = 0.1666, random_state=1)
        _,_, y_u_train[key], y_u_test[key] = train_test_split(x_inputs[key],w_u_trajs_pca[key], test_size = 0.1666, random_state=1)

In [ ]:
gpr = dict()
nn = dict()
gmr = dict()

gpr_vel = dict()
gpr_u = dict()

nn_vel = dict()
nn_u = dict()


for key in keys:
    gpr[key] = GPy_Regressor(dim_input=x_inputs_train[key].shape[1],is_transform=True)
    gpr[key].pca = pca[key]
    gpr[key].fit(x_inputs_train[key], y_train[key],num_restarts=3)
    nn[key] = NN_Regressor(K = 1)
    nn[key].fit(x_inputs_train[key], y_train[key])
    nn[key].pca = pca[key]
    #gmr[key] = BGMR_Regressor(n_components=10,n_init = 10)
    #gmr[key].fit(x_inputs_train[key], y_train[key],init_type='random')
    #gmr[key].pca = pca[key]

    if use_crocs:
        gpr_vel[key] = GPy_Regressor(dim_input=x_inputs_train[key].shape[1],is_transform=True)
        gpr_vel[key].pca = pca_vel[key]
        gpr_vel[key].fit(x_inputs_train[key], y_vel_train[key],num_restarts=3)

        nn_vel[key] = NN_Regressor(K = 1)
        nn_vel[key].pca = pca_vel[key]
        nn_vel[key].fit(x_inputs_train[key], y_vel_train[key])

        gpr_u[key] = GPy_Regressor(dim_input=x_inputs_train[key].shape[1],is_transform=True)
        gpr_u[key].pca = pca_u[key]
        gpr_u[key].fit(x_inputs_train[key], y_u_train[key],num_restarts=3)

        nn_u[key] = NN_Regressor(K = 1)
        nn_u[key].pca = pca_u[key]
        nn_u[key].fit(x_inputs_train[key], y_u_train[key])

clear_output()
    

In [ ]:
nn = dict()

for key in keys:
    nn[key] = NN_Regressor(K = 1)
    nn[key].fit(x_inputs_train[key], y_train[key])
    nn[key].pca = pca[key]

clear_output()
    

In [ ]:
functions = dict()
functions['gpr'] = gpr
functions['nn'] = nn
functions['gmr'] = gmr

if use_crocs:
    functions['gpr_vel'] = gpr_vel
    functions['gpr_u'] = gpr_u
    f = open(ROOT_CROCS + '/functions.pkl', 'wb')
else:
    f = open(ROOT_CROCS + '/functions_A.pkl', 'wb')
pickle.dump(functions, f)
f.close()

#### Load function approximators


In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'
ROOT_CROCS = '/media/teguh/Data/MEMMO Dataset/crocs_data/' + FILE_CAT + '/'
#based on database B
f = open(ROOT_CROCS + '/functions.pkl', 'rb')
functions = pickle.load(f)
f.close()
gpr = functions['gpr']
nn = functions['nn']
gmr = functions['gmr'] 
gpr_vel = functions['gpr_vel']
gpr_u = functions['gpr_u']

In [ ]:
#based on database A
f = open(ROOT_CROCS + '/functions_A.pkl', 'rb')
functions_A = pickle.load(f)
f.close()
gpr_A = functions_A['gpr']
nn_A = functions_A['nn']
gmr_A = functions_A['gmr'] 

#### Accuracy Comparison

In [ ]:
def calc_diff_pose(pose_true, pose_pred):
    diff = 0.
    for i in range(len(pose_true)):
        diff += np.linalg.norm(pose_true[i] - pose_pred[i])
    return diff

In [ ]:
#calculate time
gmr_time = []
gpr_time = []
nn_time = []

for key in keys:
    for i in range(len(x_inputs_test[key])):
        x = x_inputs_test[key][i:i+1,:]
        y_true = y_test[key][i:i+1,:]
        
        tic = time.time()
        y_pred,_ = gpr[key].predict(x)
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        toc = time.time()
        gpr_time.append(toc-tic)
        
        tic = time.time()
        y_pred,_ = nn[key].predict(x)
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        toc = time.time()
        nn_time.append(toc-tic)
        
        tic = time.time()
        y_pred,_ = gmr[key].predict(x)
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        toc = time.time()
        gmr_time.append(toc-tic)        

In [ ]:
print np.mean(gpr_time), np.mean(gmr_time),np.mean(nn_time)

In [ ]:
gmr_err = []
gpr_err = []
nn_err = []

gmr_foot_err = []
gpr_foot_err = []
nn_foot_err = []

#gpr_vel_err = []
#gpr_u_err = []

#nn_vel_err = []
#nn_u_err = []


for key in keys:
    for i in range(len(x_inputs_test[key])):
        x = x_inputs_test[key][i:i+1,:]
        y_true = y_test[key][i:i+1,:]
        traj_true = inverse_transform(y_true, pca[key], Phi, rbf_num)
        left_pose_init,right_pose_init = calc_foot_pose(traj_true[0],left_foot, right_foot, pose_type = '2D')
        left_pose_goal,right_pose_goal = calc_foot_pose(traj_true[-1],left_foot, right_foot, pose_type = '2D')
        pose_true = [left_pose_init,right_pose_init,left_pose_goal,right_pose_goal ]
        
        
        y_pred,_ = gpr[key].predict(x)
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        #gpr_err.append(np.linalg.norm(y_true-y_pred))
        gpr_err.append(np.linalg.norm(traj_true.flatten()-traj_pred.flatten()))
        left_pose_init_pred,right_pose_init_pred = calc_foot_pose(traj_pred[0],left_foot, right_foot, pose_type = '2D')
        left_pose_goal_pred,right_pose_goal_pred = calc_foot_pose(traj_pred[-1],left_foot, right_foot, pose_type = '2D')
        pose_pred = [left_pose_init_pred,right_pose_init_pred,left_pose_goal_pred,right_pose_goal_pred ]
        gpr_foot_err.append(calc_diff_pose(pose_true, pose_pred))
        
        
        
        y_pred,_ = nn[key].predict(x)
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        #nn_err.append(np.linalg.norm(y_true-y_pred))
        nn_err.append(np.linalg.norm(traj_true.flatten()-traj_pred.flatten()))
        left_pose_init_pred,right_pose_init_pred = calc_foot_pose(traj_pred[0],left_foot, right_foot, pose_type = '2D')
        left_pose_goal_pred,right_pose_goal_pred = calc_foot_pose(traj_pred[-1],left_foot, right_foot, pose_type = '2D')
        pose_pred = [left_pose_init_pred,right_pose_init_pred,left_pose_goal_pred,right_pose_goal_pred ]
        nn_foot_err.append(calc_diff_pose(pose_true, pose_pred))

        
        y_pred,_ = gmr[key].predict(x.flatten())
        traj_pred = inverse_transform(y_pred, pca[key], Phi, rbf_num)
        #gmr_err.append(np.linalg.norm(y_true-y_pred))
        gmr_err.append(np.linalg.norm(traj_true.flatten()-traj_pred.flatten()))
        left_pose_init_pred,right_pose_init_pred = calc_foot_pose(traj_pred[0],left_foot, right_foot, pose_type = '2D')
        left_pose_goal_pred,right_pose_goal_pred = calc_foot_pose(traj_pred[-1],left_foot, right_foot, pose_type = '2D')
        pose_pred = [left_pose_init_pred,right_pose_init_pred,left_pose_goal_pred,right_pose_goal_pred ]
        gmr_foot_err.append(calc_diff_pose(pose_true, pose_pred))
        
        '''
        if use_crocs:
            y_vel_true = y_vel_test[key][i:i+1,:]
            y_vel_pred,_ = gpr_vel[key].predict(x)
            gpr_vel_err.append(np.linalg.norm(y_vel_true-y_vel_pred))

            y_vel_pred,_ = nn_vel[key].predict(x)
            nn_vel_err.append(np.linalg.norm(y_vel_true-y_vel_pred))

            y_u_true = y_u_test[key][i:i+1,:]
            y_u_pred,_ = gpr_u[key].predict(x)
            gpr_u_err.append(np.linalg.norm(y_u_true-y_u_pred))

            y_u_pred,_ = nn_u[key].predict(x)
            nn_u_err.append(np.linalg.norm(y_u_true-y_u_pred))'''

In [ ]:
print "GPR & \t {0:.2f} $\\pm$ {1:.2f} \t & {2:.2f} $\\pm$ {3:.2f}".format(np.mean(gpr_err), np.std(gpr_err), np.mean(gpr_foot_err), np.std(gpr_foot_err))
print "GMR & \t {0:.2f} $\\pm$ {1:.2f} \t & {2:.2f} $\\pm$ {3:.2f}".format(np.mean(gmr_err), np.std(gmr_err), np.mean(gmr_foot_err), np.std(gmr_foot_err))
print "$k$-NN & \t {0:.2f} $\\pm$ {1:.2f} \t & {2:.2f} $\\pm$ {3:.2f}".format(np.mean(nn_err), np.std(nn_err), np.mean(nn_foot_err), np.std(nn_foot_err))

In [ ]:
if use_crocs: 
    print np.mean(gpr_vel_err),np.mean(nn_vel_err) 
    print np.std(gpr_vel_err),np.std(nn_vel_err)

    print np.mean(gpr_u_err),np.mean(nn_u_err) 
    print np.std(gpr_u_err),np.std(nn_u_err)

### Test Memory

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = dict()
for key in keys:
    gmm[key] = GaussianMixture(n_components=30, n_init = 3) 
    gmm[key].fit(x_inputs[key])

viz.set_rate(1000)

In [ ]:
indexes = [0, 3, 7, 8, 9, 12, 13]

In [ ]:
keys = ['left', 'right']

nn_A = dict()
for key in keys:
    x = gpr_A[key].gp.X
    y = gpr_A[key].gp.Y
    nn_A[key] = NN_Regressor()
    nn_A[key].fit(x,y)
    nn_A[key].pca = gpr_A[key].pca

In [ ]:
indexes = [2,4,6,8,11,14,20,21]

In [ ]:
FIG_FILE_NAME = "/home/teguh/git/publications/memmo_icra_2020/videos/gmr_"
key = 'right'
fig_index = 0
for i in (indexes):
#for i in range(50):  
    print i
    if i%2 == 0:
        key = 'left'
    else:
        key = 'right'
    x = x_inputs_test[key][np.random.randint(x_inputs_test[key].shape[0])][None,:]
    #x = x_inputs_test[key][i][None,:]
    
    #x,_ = gmm[key].sample()
    func = gpr

    #w_pca,cov = gmr[key].predict(x.flatten())
    w_pca,cov = func[key].predict(x)
    w_pca = mvn(mean=w_pca.flatten(),cov=0.01*np.eye(w_pca.shape[1])).rvs()
    w_pca[15:] *=0.
    print w_pca
    #print cov
    w = func[key].pca.inverse_transform(w_pca)
    #w = w.reshape(rbf_num,-1)
    #traj = np.dot(Phi,w)
    traj = w.reshape(3351,-1)
    if goal_type == 'contact':
        foot_marker.publish(x.reshape(-1,3))    
    elif goal_type == 'root':
        foot_marker.publish(x[0,0:6].reshape(-1,3))    
        viz.br.sendTransform(traj[0,0:3], normalize(traj[0,3:7]),
         rospy.Time.now(),
         "init",
         "world"
         )
        viz.br.sendTransform(x[0,6:9], normalize(x[0,9:13]),
                 rospy.Time.now(),
                 "goal",
                 "world"
                 )
    
    
    '''
    viz.set_dof(traj[0])
    time.sleep(0.2)
    for j,index in enumerate(np.arange(0,3350,50)):
        viz.set_dof(traj[index])
        time.sleep(0.04)
        #ave_screenshot(200,200,700,700, FIG_FILE_NAME +  str(fig_index) + '.png') 
        fig_index +=1
    for k in range(20):
        #save_screenshot(200,200,700,700, FIG_FILE_NAME  +  str(fig_index)  + '.png') 
        fig_index +=1
    '''
    
    viz.vis_traj(traj)
    #dec = raw_input()
    #if dec == 'y':
    #    indexes += [i]
    #key = toggle_key(key)
    clear_output()

In [ ]:
viz.set_dof(traj[0])

In [ ]:
for j,index in enumerate(np.arange(0,3350,50)):
    viz.set_dof(traj[index])
    time.sleep(0.04)
    save_screenshot(200,200,700,700, FIG_FILE_NAME + '_' + str(i)+ str(j) + '.png') 

clear_output()


In [ ]:
os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/gpr_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/gpr.mp4')

os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/gmr_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/gmr.mp4')

os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/nn_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/nn.mp4')

os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/gpra_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/gpr_A.mp4')

os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/gmra_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/gmr_A.mp4')

os.system('ffmpeg -r 25 -start_number 0 -i /home/teguh/git/publications/memmo_icra_2020/videos/nna_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /home/teguh/git/publications/memmo_icra_2020/videos/nn_A.mp4')

#### Getting the distribution of trajectories 

### Warmstart crocoddyl

#### Loading function approximators 

In [ ]:
#based on database B
f = open(ROOT_CROCS + '/functions.pkl', 'rb')
functions = pickle.load(f)
f.close()
gpr = functions['gpr']
nn = functions['nn']
gmr = functions['gmr'] 

gpr_vel = functions['gpr_vel']
gpr_u = functions['gpr_u']

#based on database A
f = open(ROOT_CROCS + '/functions_A.pkl', 'rb')
functions_A = pickle.load(f)
f.close()
gpr_A = functions_A['gpr']
nn_A = functions_A['nn']
gmr_A = functions_A['gmr'] 

In [ ]:
def calc_foot_T(x, move = 'left'):
    #calculate the footstep transformation matrix based on the input x
    #x is defined as: [left_foot, right_foot, 'foot_to_move']
    x = x.reshape(-1,3)
    Ts = []
    for x_i in x:
        T = PosetoMat(x_i)
        Ts.append(T)
    
    if move == 'left':
        T_lefts = [Ts[0], None, Ts[2]]
        T_rights = [Ts[1]]*3
    else:
        T_rights = [Ts[1], None, Ts[2]]
        T_lefts = [Ts[0]]*3
        
    return T_lefts, T_rights

#### Create a problem

In [ ]:
if np.random.rand() < 0.5:
    key = 'left'
else:
    key = 'right'
x,_ = gmm[key].sample()

In [ ]:
T_lefts, T_rights = calc_foot_T(x,move=key)
foot_marker.publish(x.reshape(-1,3))

#predict the trajectory
w_pca,cov = gpr[key].predict(x)
w = gpr[key].pca.inverse_transform(w_pca)
w = w.reshape(rbf_num,-1)
traj = np.dot(Phi,w)

#predict the velocity trajectory
w_pca,cov = gpr_vel[key].predict(x)
w = gpr_vel[key].pca.inverse_transform(w_pca)
w = w.reshape(rbf_num,-1)
vel_traj = np.dot(Phi,w)

#predict the control trajectory
w_pca,cov = gpr_u[key].predict(x)
w = gpr_u[key].pca.inverse_transform(w_pca)
w = w.reshape(rbf_num,-1)
u_traj = np.dot(Phi,w)


#visualize
viz.set_rate(1000)
#viz.vis_traj(traj)

#### Create phases

In [ ]:
data_phases = []
data_phases.append(np.arange(0,25))
data_phases.append(np.arange(25,60))
data_phases.append(np.arange(60,85))
delta_t = 0.04

In [ ]:
data_phases_ori = [data_phases]

#### Subsample the trajectories 

In [ ]:
T = traj.shape[0]
intervals = np.concatenate([np.arange(0,T,40),[T-1]])
T_new = len(intervals)
traj = traj[intervals]
#vel_traj = np.zeros((traj.shape[0], 38))
#u_traj = np.zeros((traj.shape[0], 32))

#for visualization
viz.set_rate(25)
raw_traj = traj.copy()

#### Store in Crocoddyl format 

In [ ]:
phases = store_for_crocoddyl(traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t)
pickle.dump(phases,open('data_teguh_3.txt','wb'))

### Run Crocoddyl 

In [ ]:
from memmo_utils import *
from crocoddyl import loadTalos
from croc_lib import *

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')
problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', num_phases=3, is_warmstart=True)
clear_output()

solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-03, recalc_u=False, TYPE_OF_SOLVER='FDDP')
traj = np.array(solver.xs)[:,0:39]

In [ ]:
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')
problem, xs, us, ts = define_croc_problem(ROBOT,'data_teguh_3.txt', num_phases=3, is_warmstart=False)
clear_output()

solver = solve_problem(ROBOT,problem,xs, us,maxiter = 50, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
traj2 = np.array(solver.xs)[:,0:39]

In [ ]:
#A quick check
#calc_foot_pose(traj[0], left_foot, right_foot)
compare_phases_with_result(phases, np.array(solver.xs), left_foot, right_foot, viz)

### Comparison between Coldstart and Warmstart 

In [ ]:
from croc_lib import *
ROBOT = loadTalos('/opt/openrobots/share/example-robot-data')


In [ ]:
data_phases_ori = []
data_phases_ori.append(np.arange(0,1001))
data_phases_ori.append(np.arange(1001,2361))
data_phases_ori.append(np.arange(2361, 3351))

viz.set_rate(1000)

In [ ]:
key = 'left'
file_name = 'data.txt'

result = dict()
res_keys = ['gpr', 'gmr', 'gpr_A', 'gmr_A','cold', 'q', 'qu', 'q_nou', 'u']
wm_methods = [gpr,gmr,gpr_A, gmr_A]
#wm_methods = []
for res_key in res_keys:
    result[res_key] = dict()
    result[res_key]['trajs'] = []
    result[res_key]['vel_trajs'] = []
    result[res_key]['u_trajs'] = []
    
    result[res_key]['n_iters'] = []
    result[res_key]['costs'] = []
    result[res_key]['x_inputs'] = []

trajs_true = []
    
#for i in range(3):

In [ ]:
key = 'right'
tic = time.time()
for i in range(len(x_inputs_test[key])):
    print 'Iteration ' + str(i)
    toc = time.time()
    print 'Time so far: ' + str(toc-tic)
    #create problem
    #x_input,_ = gmm[key].sample()
    x_input = x_inputs_test[key][i:i+1]
    foot_marker.publish(x_input.reshape(-1,3))
    w_pca_true = y_test[key][i]
    w = pca[key].inverse_transform(w_pca_true)
    w = w.reshape(rbf_num,-1)
    traj_true = np.dot(Phi,w)
    trajs_true.append(traj_true)
    
    
    #viz.vis_traj(traj_true)
    q_init = traj_true[0]
    v_init = np.zeros(38)
    
    #warmstart method
    for j, func in enumerate(wm_methods):
        res_key = res_keys[j]
        #predict the trajectory
        w_pca,cov = func[key].predict(x_input)
        w = func[key].pca.inverse_transform(w_pca)
        w = w.reshape(rbf_num,-1)
        traj = np.dot(Phi,w)

        traj_mod,_ = convert_to_croc_format(x_input,traj,key, data_phases = data_phases_ori,\
                                q_init = q_init, v_init = v_init, filename=file_name)
        print 'warmstart step' + res_key
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=3, is_warmstart=True)

        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
        traj = np.array(solver.xs)[:,0:39]
        result[res_key]['trajs'] += [traj]
        result[res_key]['n_iters'] += [solver.iter]
        result[res_key]['costs'] += [solver.cost]
        result[res_key]['x_inputs'] += [x_input]
        
    
    print 'coldstart step'
    problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=3, is_warmstart=False)

    solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=True, TYPE_OF_SOLVER='FDDP')
    traj = np.array(solver.xs)[:,0:39]
    res_key = 'cold'
    result[res_key]['trajs'] += [traj]
    result[res_key]['n_iters'] += [solver.iter]
    result[res_key]['costs'] += [solver.cost]
    

    
    #predict the trajectories
    w_pca,cov = gpr[key].predict(x_input)
    w = gpr[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    traj = np.dot(Phi,w)

    w_pca,cov = gpr_vel[key].predict(x_input)
    w = gpr_vel[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    vel_traj = np.dot(Phi,w)

    w_pca,cov = gpr_u[key].predict(x_input)
    w = gpr_u[key].pca.inverse_transform(w_pca)
    w = w.reshape(rbf_num,-1)
    u_traj = np.dot(Phi,w)

    
    wm_comps = ['q', 'qu', 'q_nou', 'u']
    for res_key in wm_comps:
        if res_key == 'q':
            inputs = [traj.copy(),None,None]
            recalc = True
        elif res_key == 'q_nou':
            inputs = [traj.copy(),None,None]
            recalc = False
        elif res_key == 'qu':
            inputs = [traj.copy(),None,u_traj]
            recalc = False
        elif res_key == 'qv':
            inputs = [traj.copy(),vel_traj,None]
            recalc = True
        elif res_key == 'qvu':
            inputs = [traj.copy(),vel_traj,u_traj]
            recalc = False
        elif res_key == 'u':
            inputs = [traj.copy()*0,None,u_traj]
            recalc = False
            
        
        traj_mod,_ = convert_to_croc_format(x_input,traj=inputs[0], vel_traj = inputs[1], \
                                u_traj = inputs[2], key = key, data_phases = data_phases_ori,\
                                q_init = q_init, v_init = v_init, filename=file_name)
        print 'warmstart step' + res_key
        problem, xs, us, ts = define_croc_problem(ROBOT,file_name, num_phases=3, is_warmstart=True)

        solver = solve_problem(ROBOT,problem,xs, us,maxiter = 20, STOP_THRESHOLD=1e-02, recalc_u=recalc, TYPE_OF_SOLVER='FDDP')
        res_traj = np.array(solver.xs)[:,0:39]
        result[res_key]['trajs'] += [res_traj]
        result[res_key]['vel_trajs'] += [np.array(solver.xs)[:,39:]]
        result[res_key]['u_trajs'] += [np.array(solver.us)]
        result[res_key]['n_iters'] += [solver.iter]
        result[res_key]['costs'] += [solver.cost]
        result[res_key]['x_inputs'] += [x_input]
    
    
    
    
    clear_output()
    success = dict()
    for res_key in res_keys:
        success[res_key] = np.array(result[res_key]['costs']) < 1e2
        print"{0} \t & {1:.2f} & \t {2:.2f} $\\pm${3:.2f} & \t {4:.2f} $\\pm$ {5:.2f}".format(res_key,  100.*np.sum(success[res_key])/(len(success[res_key])+.000001), np.mean(np.array(result[res_key]['costs'])[success[res_key]]), np.std(np.array(result[res_key]['costs'])[success[res_key]]), np.mean(np.array(result[res_key]['n_iters'])[success[res_key]]),np.std(np.array(result[res_key]['n_iters'])[success[res_key]]))

In [ ]:
success = dict()
for res_key in res_keys:
    success[res_key] = np.array(result[res_key]['costs']) < 1e2
    print"{0} \t & {1:.2f} & \t {2:.2f} $\\pm${3:.2f} & \t {4:.2f} $\\pm$ {5:.2f}".format(res_key,  100.*np.sum(success[res_key])/(len(success[res_key])+.000001), np.mean(np.array(result[res_key]['costs'])[success[res_key]]), np.std(np.array(result[res_key]['costs'])[success[res_key]]), np.mean(np.array(result[res_key]['n_iters'])[success[res_key]]),np.std(np.array(result[res_key]['n_iters'])[success[res_key]]))

In [ ]:
f = open(ROOT_CROCS + '/result_single_leftright.pkl', 'wb')
pickle.dump(result,f)
f.close()